# 英语完形填空

## 准备工作

In [6]:
import keras
from keras.layers import Input, Embedding, Bidirectional, Dense, LSTM, Dropout
from keras.models import Model, Sequential
from keras.optimizers import Adam, SGD
import numpy as np


MAX_WORD = 2150000
WORD_OUT = 4000
print("done")

Using TensorFlow backend.


done


In [7]:
with open("d:/Data/lexicon1.txt", encoding="utf-8") as f:
    lexicon = f.readlines()
with open("d:/Data/coca-samples-db/db_mag.txt") as f:
    db_mag = f.readlines()

#### 预处理3500词

In [8]:
with open("d:/Data/3500.txt") as f:
    w35 = f.readlines()
word35 = ["##UNK"]
for line in w35:
    l = line.split()
    if len(l) < 3:
        continue
    word35.append(l[0])
word35id = {w:Id for Id, w in enumerate(word35)}
print(len(word35), word35[:10], word35id["abnormal"])

3857 ['##UNK', 'abandon', 'ability', 'able', 'abnormal', 'aboard', 'abolish', 'abortion', 'about', 'above'] 4


#### 定义解码函数
用于将id化的句子变为可读的句子

In [9]:
def decode(sentence):
    out = ""
    for word in sentence:
        out += id2word[word] + " "
    return out

#### 预处理句子

In [10]:
def preprocess(file):
    Id = 0
    ids = []
    sentence = ()
    sentences = []
    for line in file:
        l = line.split()
        Id_this = l[0]
        if Id != Id_this:
            sentences.append(sentence)
            sentence = ()
            Id = Id_this
            ids.append(Id)
        sentence += (l[2],)
    return sentences, ids


sentences, ids = preprocess(db_mag)
print(len(sentences))

948


In [11]:
a = [len(sentences[i]) for i in range(948)]
print(max(a))
print(sum(a)/948)

11756
1652.714135021097


#### 预处理输入词典

In [19]:
def vulcab(file):
    id2word = ["##UNK"]
    word2id = {"##UNK":0}
    oldId2id = [0]
    word2org = {"##UNK":"##UNK"}
    for i, line in enumerate(file):
        if not line.strip():
            break
        word = line.split()
        if len(word) != 4:
            oldId2id.append(0)
            continue
        oldId, w, org, _ = word
        w = w.lower()
        if w in word2id:
            oldId2id.append(word2id[w])
            continue
        if oldId != str(len(oldId2id)):
            for j in range(int(oldId)-len(oldId2id)):
                oldId2id.append(0)
        word2id[w] = len(id2word)
        oldId2id.append(len(id2word))
        id2word.append(w)
        word2org[w] = org
    return id2word, word2id, oldId2id, word2org

id2word, word2id, oldId2id, word2org = vulcab(lexicon)
print(word2id["like"])

print(len(word2id), len(id2word))
print(id2word[word2id["interest"]], id2word[oldId2id[333791]])

89
2149218 2149218
interest corda


In [20]:
print([word2id["it"]], oldId2id[54])

[14] 14


#### 根据新的单词id，再次处理读取后的句子

In [23]:
new_s = [tuple(oldId2id[int(word)] for word in sentence if id2word[oldId2id[int(word)]] not in ("<p>", "//", "@") )for sentence in sentences]

print(decode(new_s[250]))

##UNK section : the mcdonald 's of fitness his goal : to single-handedly save the world 's women from obesity . with curves , he seems to be pulling it off . and he 's built a billion-dollar business along the way it 's just after 9 in the morning , and 375 people , nearly all of them women , sit at rows of tables in quiet anticipation . in front of each is a bottle of water , a thick loose-leaf binder , and a paper tent printed with their names and hometowns : boynton beach , florida ; kodiak , alaska ; libby , montana ; traralgon , australia ; martha 's vineyard ; san diego . the room 's walls are draped with large boxing match posters -- promoting contests like iron will vs . evil sofa , and will power vs . curly fries . a large neon sign of the logo for the women 's fitness club curves -- the company name , in purple cursive -- is mounted on the wall , casting a violet glow on the white metal weight machines that occupy a pride of place not only in ##UNK ##UNK ##UNK ##UNK ##UNK ##U

## 定义模型

In [ ]:
model = Sequential()
model.add(Embedding(128, input_shape=(MAX_WORD,)))
model.add(Bidirectional(LSTM(50, return_sequence=True)))
model.add(Bidirectional(LSTM(50)))
model.add(Dense(WORD_OUT, activation="softmax"))

In [ ]:
np.save("")